In [1]:
from __future__ import print_function

import os
import json

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd

In [ ]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)
torch.autograd.set_detect_anomaly(True)

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig
from trl import SFTTrainer 

In [4]:
base_llm_id = "microsoft/phi-2"

In [22]:
lora_config = LoraConfig(
    lora_alpha=64,
    lora_dropout=0.05,
    r=32,
    target_modules= ['Wqkv', 'fc1', 'fc2'], # Set the target modules to be loaded in LoRA
    bias="none",
    task_type="CAUSAL_LM", 
)

In [6]:
quant_config = BitsAndBytesConfig(load_in_8bit=True, 
                                  bnb_8bit_quant_type='nf8',
                                    bnb_8bit_compute_dtype=torch.bfloat16, 
                                    bnb_8bit_use_double_quant=False)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(base_llm_id, 
                                                trust_remote_code=True, 
                                                quantization_config=quant_config,
                                                low_cpu_mem_usage=True,
                                                flash_attn = True,
                                                flash_rotary = True,
                                                fused_dense = True,
                                                revision = 'refs/pr/23')

In [24]:
model.config.use_cache = False
model = get_peft_model(model, lora_config)


In [ ]:
model.print_trainable_parameters()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_llm_id,
    padding_side="left",
    use_fast=True, 
)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id
model.resize_token_embeddings(len(tokenizer))

In [11]:
data = pd.read_excel('~/Downloads/SC2.xlsx')

In [ ]:
data.head()

In [13]:
def create_prompt(data):
    prompt = f"### Question: {data['instruction']}\n ### Answer: {data['response']}"
    return prompt

In [14]:
from datasets  import Dataset
train_data = Dataset.from_pandas(data)

In [ ]:
train_data

In [16]:
def tokenize(data):
    result = tokenizer(create_prompt(data), 
                     return_tensors="pt", 
                     max_length=820, 
                     padding='max_length', 
                     truncation=True)
    result['labels'] = result['input_ids']
    return result

In [ ]:
tokenized_train_dataset = train_data.map(tokenize, batched=True, batch_size=1, remove_columns=['instruction', 'response'])

In [ ]:
tokenized_train_dataset

In [ ]:
project = "sc-finetune"
base_model_name = "phi2"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

trainer = SFTTrainer( #transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    packing=True,
    max_seq_length=820,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=80,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        fp16=False,
        bf16=True,
        weight_decay=0.01,  # in order to
        #max_steps=1000,
        learning_rate=2.5e-5, # Want a small lr for finetuning
        optim="adamw_8bit",
        logging_steps=100,              # When to start reporting loss
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=880,                # Save checkpoints every 50 steps
        #evaluation_strategy  = "steps",
        #eval_steps=100,               # Evaluate every 50 steps
        
        lr_scheduler_type='cosine',
        report_to="none",
        num_train_epochs=160,
        #load_best_model_at_end=True,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
#model.config.ctc_zero_infinity = True

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('/data/MSC-master/phi2-sc-finetune/checkpoint-880/')

In [24]:
trainer.save_state()

In [ ]:
trainer.train(resume_from_checkpoint='/data/MSC-master/phi2-sc-finetune/checkpoint-800/')

In [ ]:
new_model = ft_model.merge_and_unload()
new_model.save_pretrained('scgpt_stage1', save_config=True)
tokenizer.save_pretrained('scgpt_stage1')

In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_llm_id,
    #add_bos_token=True,
    trust_remote_code=True,
    use_fast=True,
)

In [ ]:

eval_prompt = f"### Question: What are the available Build_ actions for Terran?\n ### Answer: "

model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")
#ft_model.eval()
model.eval()
#ft_model.to("cuda")
with torch.no_grad():
    gen = model.generate(**model_input, max_new_tokens=100)[0]
    print(tokenizer.decode(gen, skip_special_tokens=True))